In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
train_ds = tfds.load('malaria', split='train[:80%]', as_supervised=True)
validation_ds = tfds.load('malaria', split='train[80%:]', as_supervised=True)

# Preprocess the dataset
def preprocess_image(image, label):
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
validation_ds = validation_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Batch and prefetch the datasets
train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
validation_ds = validation_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')  # 2 classes: uninfected and parasitized
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, epochs=10, validation_data=validation_ds)

# Evaluate the model
test_ds = tfds.load('malaria', split='test', as_supervised=True)
test_ds = test_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
loss, accuracy = model.evaluate(test_ds)
print(f"Validation accuracy: {accuracy:.2f}")

# Function to preprocess a single new image
def preprocess_single_image(image):
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Predict on a new image (assuming new_image is loaded as a tf.Tensor)
# Example: new_image = tf.io.decode_image(tf.io.read_file('path_to_image.jpg'))
new_image = ...  # Load your own blood smear image
new_image = preprocess_single_image(new_image)
prediction = model.predict(new_image)
predicted_class = tf.argmax(prediction[0]).numpy()

if predicted_class == 1:
    print("Parasitized")
else:
    print("Uninfected")


# Train the model
model.fit(ds, epochs=50, callbacks=[early_stopping])